## SCRIPT PARA EXTRAER ETAPA F8 DE ENLACE https://ofi5.mef.gob.pe/invierte/ejecucion/verFichaEjecucion/2577906

In [3]:
import pandas as pd
import time
import os
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ================================================================
# CONFIGURACIÓN OPTIMIZADA
# ================================================================
RUTA_ENTRADA = r"C:\Users\user\Downloads\CUI_junto.xlsx"
RUTA_SALIDA = r"C:\Users\user\Downloads\ETAPA CUI MINSA1.xlsx"
MAX_REINTENTOS = 2
MODO_VISIBLE = True
GUARDAR_CADA = 5

# Timeouts optimizados
TIMEOUT_PAGINA = 20
TIMEOUT_ELEMENTO = 10

# ================================================================
# FUNCIONES DE CHECKPOINT MEJORADAS
# ================================================================
def cargar_progreso():
    """Carga progreso previo si existe"""
    if os.path.exists(RUTA_SALIDA):
        try:
            df = pd.read_excel(RUTA_SALIDA)
            print(f"📥 Progreso encontrado: {len(df)} CUIs ya procesados")
            return df.to_dict('records')
        except:
            pass
    return []

def obtener_pendientes(completa, procesados):
    """Calcula CUIs pendientes O con NO DISPONIBLE"""
    if not procesados:
        print(f"📋 Todos los CUIs pendientes: {len(completa)}")
        return completa, []
    
    # Crear diccionario de procesados
    procesados_dict = {str(r['CUI']): r for r in procesados}
    
    # 1. CUIs nunca procesados
    cuis_nuevos = [cui for cui in completa if str(cui) not in procesados_dict]
    
    # 2. CUIs con "NO DISPONIBLE" en Etapa
    cuis_reintentar = []
    for cui in completa:
        if str(cui) in procesados_dict:
            registro = procesados_dict[str(cui)]
            etapa = str(registro.get('Etapa', '')).strip().upper()
            
            # Si tiene "NO DISPONIBLE" en la etapa, reintentarlo
            if etapa == 'NO DISPONIBLE' or etapa == '':
                cuis_reintentar.append(cui)
    
    # Combinar ambos
    pendientes = cuis_nuevos + cuis_reintentar
    
    if pendientes:
        print(f"📋 Nuevos: {len(cuis_nuevos)} | Reintentos: {len(cuis_reintentar)} | Total pendiente: {len(pendientes)}")
    
    return pendientes, procesados_dict

def guardar(resultados):
    """Guarda progreso"""
    try:
        pd.DataFrame(resultados).to_excel(RUTA_SALIDA, index=False)
    except Exception as e:
        print(f"⚠️ Error guardando: {e}")

# ================================================================
# LEER CUIs
# ================================================================
print("📂 Cargando CUIs...")
df_cui = pd.read_excel(RUTA_ENTRADA)
lista_completa = df_cui['CUI'].astype(str).tolist()

resultados_previos = cargar_progreso()
lista_cuis, procesados_dict = obtener_pendientes(lista_completa, resultados_previos)

if not lista_cuis:
    print("🎉 ¡Todos los CUIs ya fueron procesados exitosamente!")
    print("💡 Si quieres reintentar CUIs fallidos, verifica que tengan 'NO DISPONIBLE' en el archivo.")
    exit()

# Inicializar resultados con los previos (para mantener los exitosos)
resultados = resultados_previos.copy()

# ================================================================
# CONFIGURAR NAVEGADOR OPTIMIZADO
# ================================================================
def crear_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()

    if not MODO_VISIBLE:
        options.add_argument("--headless=new")
    else:
        options.add_argument("--start-maximized")

    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-logging")
    options.add_argument("--log-level=3")
    options.add_argument("--silent")

    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2,
        "profile.default_content_setting_values.media_stream": 2,
        "profile.default_content_setting_values.geolocation": 2,
    }
    options.add_experimental_option("prefs", prefs)
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.page_load_strategy = 'normal'

    driver = Chrome(service=service, options=options)
    driver.set_page_load_timeout(TIMEOUT_PAGINA)
    driver.set_script_timeout(15)
    return driver

driver = crear_driver()
print(f"⚡ Procesando {len(lista_cuis)} CUIs pendientes\n")

# ================================================================
# EXTRACCIÓN DE ETAPA
# ================================================================
def extraer_etapa(driver):
    """Extrae solo el valor de ETAPA desde el HTML"""
    try:
        etapa = driver.execute_script("""
            try {
                // Buscar el div con el estilo específico
                var divs = document.querySelectorAll('div[style*="background-color: #D9EDF7"]');
                
                for (var i = 0; i < divs.length; i++) {
                    var texto = divs[i].textContent || divs[i].innerText || '';
                    
                    // Buscar el patrón "ETAPA:" seguido de un <strong>
                    if (texto.includes('ETAPA:')) {
                        var strong = divs[i].querySelector('strong');
                        if (strong) {
                            var valor = strong.textContent.trim();
                            if (valor && valor !== '') {
                                return valor;
                            }
                        }
                    }
                }
                
                // Método alternativo: buscar directamente por el patrón de texto
                var allStrongs = document.querySelectorAll('strong');
                for (var j = 0; j < allStrongs.length; j++) {
                    var prev = allStrongs[j].previousSibling;
                    if (prev && prev.textContent && prev.textContent.includes('ETAPA:')) {
                        return allStrongs[j].textContent.trim();
                    }
                }
                
                return 'NO DISPONIBLE';
            } catch(e) {
                console.error('Error extrayendo etapa:', e);
                return 'NO DISPONIBLE';
            }
        """)
        
        if etapa and etapa != 'NO DISPONIBLE':
            return etapa
        else:
            print(f"⚠️ Etapa no encontrada", end=" ")
            return 'NO DISPONIBLE'

    except Exception as e:
        print(f"⚠️ Error: {str(e)[:20]}", end=" ")
        return 'NO DISPONIBLE'

# ================================================================
# PROCESAR CUI
# ================================================================
def procesar_cui(cui, intento=1):
    try:
        # Construir la URL directa
        url = f"https://ofi5.mef.gob.pe/invierte/ejecucion/verFichaEjecucion/{cui}"
        driver.get(url)
        
        # Esperar a que cargue el contenido
        Wait(driver, TIMEOUT_ELEMENTO).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        time.sleep(0.5)  # Pequeña pausa para asegurar carga completa

        # Extraer la etapa
        etapa = extraer_etapa(driver)
        
        if etapa == 'NO DISPONIBLE':
            raise Exception("Etapa no disponible")

        return {'etapa': etapa}

    except Exception as e:
        if intento < MAX_REINTENTOS:
            print(f"🔄{intento+1}...", end=" ")
            time.sleep(1)
            return procesar_cui(cui, intento + 1)
        else:
            raise e

# ================================================================
# ACTUALIZAR O AGREGAR RESULTADO
# ================================================================
def actualizar_resultado(cui, datos):
    """Actualiza un CUI existente o agrega uno nuevo"""
    # Buscar si ya existe en resultados
    for i, r in enumerate(resultados):
        if str(r['CUI']) == str(cui):
            # ACTUALIZAR el existente
            resultados[i] = {
                "CUI": cui,
                "Etapa": datos['etapa']
            }
            return
    
    # Si no existe, AGREGAR nuevo
    resultados.append({
        "CUI": cui,
        "Etapa": datos['etapa']
    })

def agregar_fallo(cui):
    """Agrega o mantiene un CUI como fallido"""
    for r in resultados:
        if str(r['CUI']) == str(cui):
            return  # Ya existe con NO DISPONIBLE, no hacer nada
    
    # Agregar nuevo con NO DISPONIBLE
    resultados.append({
        "CUI": cui,
        "Etapa": "NO DISPONIBLE"
    })

# ================================================================
# LOOP PRINCIPAL
# ================================================================
tiempo_inicio = time.time()
exitosos = 0
fallos = 0

print(f"{'='*60}")
print(f"Inicio: {time.strftime('%H:%M:%S')}")
print(f"{'='*60}\n")

try:
    for idx, cui in enumerate(lista_cuis, 1):
        t_inicio = time.time()
        
        # Verificar si es reintento
        es_reintento = str(cui) in procesados_dict
        marca = "🔄" if es_reintento else "🆕"
        
        print(f"{marca} [{idx}/{len(lista_cuis)}] {cui}:", end=" ")
        
        try:
            datos = procesar_cui(cui)
            actualizar_resultado(cui, datos)
            exitosos += 1
            print(f"✅ {datos['etapa']} ({time.time()-t_inicio:.1f}s)")
            
        except Exception as e:
            agregar_fallo(cui)
            fallos += 1
            print(f"❌ ({time.time()-t_inicio:.1f}s)")

        # GUARDAR CADA N CUIs
        if (idx % GUARDAR_CADA == 0) or (idx == len(lista_cuis)):
            guardar(resultados)
            if idx % GUARDAR_CADA == 0:
                print(f"   💾", end="")

        # Progreso cada 10 CUIs
        if idx % 10 == 0:
            t = time.time() - tiempo_inicio
            prom = t / idx
            eta = (len(lista_cuis) - idx) * prom
            print(f"\n   📊 {exitosos}✅ {fallos}❌ | {t:.0f}s | ETA:{eta:.0f}s\n")

except KeyboardInterrupt:
    print("\n\n⚠️ INTERRUMPIDO")
    guardar(resultados)
    driver.quit()
    print(f"💾 Progreso guardado. Ejecuta nuevamente para continuar.")
    exit()

# ================================================================
# FINALIZAR
# ================================================================
driver.quit()
guardar(resultados)

tiempo_total = time.time() - tiempo_inicio

print(f"\n{'='*60}")
print("🏁 COMPLETADO")
print(f"{'='*60}")
print(f"📊 Total procesados en sesión: {len(lista_cuis)}")
print(f"✅ Exitosos en sesión: {exitosos}")
print(f"❌ Fallidos en sesión: {fallos}")
print(f"⏱️ Tiempo: {tiempo_total:.1f}s ({tiempo_total/60:.1f} min)")
print(f"💾 Archivo: {RUTA_SALIDA}")

# Estadísticas finales del archivo completo
df_final = pd.DataFrame(resultados)
total_exitosos = len(df_final[df_final['Etapa'] != 'NO DISPONIBLE'])
total_fallidos = len(df_final[df_final['Etapa'] == 'NO DISPONIBLE'])

print(f"\n📊 ESTADÍSTICAS TOTALES DEL ARCHIVO:")
print(f"   Total CUIs: {len(df_final)}")
print(f"   ✅ Con datos: {total_exitosos} ({total_exitosos/len(df_final)*100:.1f}%)")
print(f"   ❌ Sin datos: {total_fallidos} ({total_fallidos/len(df_final)*100:.1f}%)")
print(f"{'='*60}")

📂 Cargando CUIs...
📋 Todos los CUIs pendientes: 937
⚡ Procesando 937 CUIs pendientes

Inicio: 23:45:04

🆕 [1/937] 2507146: ✅ Aprobación de consistencia o modificaciones antes de la aprobación del ET o ET (A) (1.0s)
🆕 [2/937] 2654533: ⚠️ Etapa no encontrada 🔄2... ⚠️ Etapa no encontrada ❌ (2.2s)
🆕 [3/937] 2690987: ⚠️ Etapa no encontrada 🔄2... ⚠️ Etapa no encontrada ❌ (2.3s)
🆕 [4/937] 2638397: ⚠️ Etapa no encontrada 🔄2... ⚠️ Etapa no encontrada ❌ (2.3s)
🆕 [5/937] 2414624: ✅ Aprobación de consistencia o modificaciones antes de la aprobación del ET o ET (A) (0.8s)
   💾🆕 [6/937] 2094808: ✅ Ejecución física (C) (1.0s)
🆕 [7/937] 2573274: ✅ Aprobación de consistencia o modificaciones antes de la aprobación del ET o ET (A) (0.8s)
🆕 [8/937] 2234314: ✅ Aprobación de consistencia o modificaciones antes de la aprobación del ET o ET (A) (0.8s)
🆕 [9/937] 2697841: ⚠️ Etapa no encontrada 🔄2... ⚠️ Etapa no encontrada ❌ (2.3s)
🆕 [10/937] 2260211: ✅ Ejecución física (C) (0.9s)
   💾
   📊 6✅ 4❌ | 14s | ETA:1